In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, confusion_matrix, 
    precision_score, recall_score, 
    roc_curve, auc
)
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [8]:
#Denne funktion bestemmer hvilket label "DelayLabel" ender på basseret på forsinkelsen 
def label_delay(delay):
    if delay <= 15:
        return 'on-time'
    elif delay <= 120:
        return 'late'
    else:
        return 'very-late'

In [9]:
#Henter vores datasæt og laver det til pandas dataframe
df = pd.read_csv('Combined_Flights_2022.csv', nrows = 900000)

In [10]:
#DelayLabel bliver tilføjet og apply bruger funktionen label_delay på hele rækken
df['DelayLabel'] = df['ArrDelayMinutes'].apply(label_delay)

In [11]:
#Definere de kolonner vi gerne vil træne på
relevant_columns = ['Airline', 'Origin', 'Dest', 
                    'DepTime', 'ArrTime', 'DelayLabel', 
                    'Distance', 'DayOfWeek', 'DayofMonth', 'Quarter']

#Beholder kun de data vi vil træne på
df = df[relevant_columns]

In [12]:
#One-hot encoder vores koloner
df = pd.get_dummies(df, columns=['Airline', 'Origin', 'Dest'], dtype=int)

In [13]:
#skalere vores koloner
scaler = StandardScaler()
columns_to_normalize = ["DepTime", "ArrTime", 'Distance']
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

In [14]:
# fjerner alle rækker med tomme felter
rows_before = len(df)
df.dropna(inplace=True)
rows_after = len(df)
rows_removed = rows_before - rows_after
print(f"Fjernet {rows_removed} rækker.")

Fjernet 27717 rækker.


In [15]:
#fjerne DelayLabel fra df og gemmer dem som label
label = df.pop("DelayLabel")

In [16]:
#Laver et 80/20 split på vores data og labels
train_x, test_x, train_y, test_y = train_test_split(df, label, stratify=label, test_size=0.20, random_state=1)

In [17]:
#definere og fitter vores model
dtc = RandomForestClassifier()
model = dtc.fit(train_x, train_y)


In [18]:
#forudsiger labels på vores test sæt
predicted_values = dtc.predict(test_x)

In [19]:
#sammenligner vores test resultater og vores predicted_values
acc = accuracy_score(test_y, predicted_values)
conf_matrix = confusion_matrix(test_y, predicted_values)
prec = precision_score(test_y, predicted_values, average='weighted')
rec = recall_score(test_y, predicted_values, average='weighted')

In [20]:
# Forudsige sandsynligheder for test-sættet
predicted_probabilities = dtc.predict_proba(test_x)

# Udskrive de forudsagte sandsynligheder for de første 5 test-eksempler
print(predicted_probabilities[:5])

[[0.15 0.83 0.02]
 [0.02 0.98 0.  ]
 [0.06 0.94 0.  ]
 [0.1  0.9  0.  ]
 [0.16 0.79 0.05]]


In [21]:
print(f"Accuracy: {acc}")
print(f"Precision: {prec}")
print(f"Recall: {rec}")

classes = model.classes_
print("Confusion Matrix:")
print(f"Labels: {classes}")
print(conf_matrix)

Accuracy: 0.841210154937893
Precision: 0.8221844016589938
Recall: 0.841210154937893
Confusion Matrix:
Labels: ['late' 'on-time' 'very-late']
[[  9338  20785    279]
 [  2286 136698     90]
 [  1641   2621    719]]
